In [ ]:
import triangle
import numpy as np
from pprint import pprint
import matplotlib.pyplot as plt
from matplotlib.tri import Triangulation

%matplotlib notebook

In [ ]:
def triangle_x(m):
    return m['vertices'][m['triangles'],0]

def triangle_y(m):
    return m['vertices'][m['triangles'],1]

def triangle_cog_x(m):
    return np.mean(triangle_x(m), axis=1)

def triangle_cog_y(m):
    return np.mean(triangle_y(m), axis=1)

def triangle_cog(m):
    return np.hstack((triangle_cog_x(m)[:,None], triangle_cog_y(m)[:,None]))

def plot_mesh(m, vertex_numbers=False, triangle_numbers=False, edge_numbers=False, edge_labels=False):
    plt.triplot(m['vertices'][:,0], m['vertices'][:,1], m['triangles'])
    
    if vertex_numbers:
        for i in range(m['vertices'].shape[0]):
            plt.text(m['vertices'][i, 0], m['vertices'][i, 1], str(i),
                     color='r',
                     horizontalalignment='center',
                     verticalalignment='center')
            
    if triangle_numbers:
        cogs = triangle_cog(m)
        for i in range(cogs.shape[0]):
            plt.text(cogs[i, 0], cogs[i, 1], str(i))
            
    if edge_labels or edge_numbers:
        from edges import meshEdges
        edge, edge_markers, ElementEdges = meshEdges(m)
        
    if edge_numbers:
        for i in range(edge.shape[0]):
            _x = np.mean(m['vertices'][edge[i,:], 0])
            _y = np.mean(m['vertices'][edge[i,:], 1])
            plt.text(_x, _y, str(i), color='g')
            
    if edge_labels:
        for i in range(edge.shape[0]):
            _x = np.mean(m['vertices'][edge[i,:], 0])
            _y = np.mean(m['vertices'][edge[i,:], 1])
            plt.text(_x, _y, edge_markers[i,0], color='g')

### Question 1

In [ ]:
# input: numbers start at 1
g = {
    'vertices': np.copy(np.array([[0, 3, 3, 0], [0, 0, 1.5, 1.5]]).T),
    'segments': np.copy(np.array([[0, 1, 2, 3], [1, 2, 3, 0]]).T)
}

In [ ]:
# index starts at 0
m1 = triangle.triangulate(g)
pprint(m1)

### Question 2

In [ ]:
maxArea = 0.125
m2 = triangle.triangulate(g,"pDa"+str(maxArea))
print("Number triangles = {}".format(m2['triangles'].shape[0]))

### Question 3

In [ ]:
plt.figure()
plot_mesh(m2, vertex_numbers=True, triangle_numbers=True, edge_numbers=True, edge_labels=False)
plt.axis("equal")
plt.show()

### Question 4

In [ ]:
maxArea = 0.2
m3 = triangle.triangulate(g,"Da"+str(maxArea))
area = 0.2 * (np.mean(triangle_x(m3), axis=1) ** 2)
m3['triangle_max_area']=area
m4=triangle.triangulate(m3,'Dra')
print("Number triangles before = {}".format(m3['triangles'].shape[0]))
print("Number triangles after  = {}".format(m4['triangles'].shape[0]))

In [ ]:
plt.figure()
plot_mesh(m4, vertex_numbers=True, triangle_numbers=True)
plt.axis("equal")
plt.show()

### Question 5

In [ ]:
def geom_disc(n=10):
    vert_outer = np.copy(np.array([[-1.0, 1, 1, -1],[-1, -1, 1, 1]]).T)
    seg_outer = np.copy(np.array([[0, 1, 2, 3], [1, 2, 3, 0]]).T)
    
    r_circle = 0.5
    t_circle = np.linspace(0, 2*np.pi, n, endpoint=False)
    x_circle = r_circle*np.cos(t_circle)
    y_circle = r_circle*np.sin(t_circle)
    
    vert_inner = np.hstack((x_circle[:, None], y_circle[:, None]))
    ind_inner = np.arange(n)+4
    seg_inner = np.hstack((ind_inner[:, None], np.roll(ind_inner, -1)[:, None]))
    
    g = {
        'vertices': np.vstack((vert_outer, vert_inner)),
        'segments': np.vstack((seg_outer, seg_inner)),
        'holes': np.zeros((1,2)),
    } 
    
    m_ext = triangle.triangulate(g, 'Dpqa0.02s')
    
    plt.figure()
    plot_mesh(m_ext, vertex_numbers=True, triangle_numbers=False)
    plt.axis("equal")
    plt.show()
    
    return m_ext

In [ ]:
m = geom_disc(20)